In [2]:
import pandas as pd

In [3]:
file = "data/dataset_with_sentiment.json"
df = pd.read_json(file)
print(df.columns)
df["sentiment_siebert/sentiment-roberta-large-english"] = df["sentiment_siebert/sentiment-roberta-large-english"].str.lower()
df.head()

Index(['text', 'drivers',
       'sentiment_lxyuan/distilbert-base-multilingual-cased-sentiments-student',
       'sentiment_finiteautomata/beto-sentiment-analysis',
       'sentiment_cardiffnlp/twitter-roberta-base-sentiment',
       'sentiment_cardiffnlp/twitter-roberta-base-sentiment-latest',
       'sentiment_cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual',
       'sentiment_Cloudy1225/stackoverflow-roberta-base-sentiment',
       'sentiment_nlptown/bert-base-multilingual-uncased-sentiment',
       'sentiment_Moritz-Pfeifer/CentralBankRoBERTa-sentiment-classifier',
       'sentiment_siebert/sentiment-roberta-large-english'],
      dtype='object')


,text,drivers,sentiment_lxyuan/distilbert-base-multilingual-cased-sentiments-student,sentiment_finiteautomata/beto-sentiment-analysis,sentiment_cardiffnlp/twitter-roberta-base-sentiment,sentiment_cardiffnlp/twitter-roberta-base-sentiment-latest,sentiment_cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual,sentiment_Cloudy1225/stackoverflow-roberta-base-sentiment,sentiment_nlptown/bert-base-multilingual-uncased-sentiment,sentiment_Moritz-Pfeifer/CentralBankRoBERTa-sentiment-classifier,sentiment_siebert/sentiment-roberta-large-english
0,This why Ricc career is over... we see again t...,[Lance Stroll],negative,neutral,negative,negative,negative,negative,negative,negative,negative
1,0:19,[],positive,neutral,neutral,neutral,neutral,neutral,negative,positive,negative
2,Hall Cynthia Jackson Kimberly Lewis Sharon,[Lewis Hamilton],positive,neutral,neutral,neutral,neutral,neutral,positive,positive,positive
3,🔥,[],positive,neutral,positive,positive,positive,positive,positive,positive,positive
4,Lee Ronald Hall Brenda Moore Jeffrey,[],positive,neutral,neutral,neutral,neutral,neutral,positive,positive,positive


In [4]:
def print_values(df):
    for column in df.columns[2:]:
        print(f"Value counts for column '{column}':")
        print(df[column].value_counts())
        print("-" * 40)
print_values(df)

Value counts for column 'sentiment_lxyuan/distilbert-base-multilingual-cased-sentiments-student':
sentiment_lxyuan/distilbert-base-multilingual-cased-sentiments-student
positive    45390
negative    32956
neutral      4698
Name: count, dtype: int64
----------------------------------------
Value counts for column 'sentiment_finiteautomata/beto-sentiment-analysis':
sentiment_finiteautomata/beto-sentiment-analysis
neutral     61310
negative    12682
positive     9052
Name: count, dtype: int64
----------------------------------------
Value counts for column 'sentiment_cardiffnlp/twitter-roberta-base-sentiment':
sentiment_cardiffnlp/twitter-roberta-base-sentiment
neutral     35858
negative    24373
positive    22813
Name: count, dtype: int64
----------------------------------------
Value counts for column 'sentiment_cardiffnlp/twitter-roberta-base-sentiment-latest':
sentiment_cardiffnlp/twitter-roberta-base-sentiment-latest
neutral     32143
negative    25968
positive    24933
Name: count, 

In [7]:
# df.drop(columns=["sentiment_finiteautomata/beto-sentiment-analysis", "sentiment_Moritz-Pfeifer/CentralBankRoBERTa-sentiment-classifier", "sentiment_siebert/sentiment-roberta-large-english"], inplace=True)
print_values(df)

Value counts for column 'sentiment_lxyuan/distilbert-base-multilingual-cased-sentiments-student':
sentiment_lxyuan/distilbert-base-multilingual-cased-sentiments-student
positive    45390
negative    32956
neutral      4698
Name: count, dtype: int64
----------------------------------------
Value counts for column 'sentiment_cardiffnlp/twitter-roberta-base-sentiment':
sentiment_cardiffnlp/twitter-roberta-base-sentiment
neutral     35858
negative    24373
positive    22813
Name: count, dtype: int64
----------------------------------------
Value counts for column 'sentiment_cardiffnlp/twitter-roberta-base-sentiment-latest':
sentiment_cardiffnlp/twitter-roberta-base-sentiment-latest
neutral     32143
negative    25968
positive    24933
Name: count, dtype: int64
----------------------------------------
Value counts for column 'sentiment_cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual':
sentiment_cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual
negative    31438
positive 

In [8]:
df_selected = df.iloc[:, 2:]
df.loc[:, 'sentiment_groundtruth'] = df_selected.mode(axis=1)[0]
df.head()

,text,drivers,sentiment_lxyuan/distilbert-base-multilingual-cased-sentiments-student,sentiment_cardiffnlp/twitter-roberta-base-sentiment,sentiment_cardiffnlp/twitter-roberta-base-sentiment-latest,sentiment_cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual,sentiment_Cloudy1225/stackoverflow-roberta-base-sentiment,sentiment_nlptown/bert-base-multilingual-uncased-sentiment,sentiment_groundtruth
0,This why Ricc career is over... we see again t...,[Lance Stroll],negative,negative,negative,negative,negative,negative,negative
1,0:19,[],positive,neutral,neutral,neutral,neutral,negative,neutral
2,Hall Cynthia Jackson Kimberly Lewis Sharon,[Lewis Hamilton],positive,neutral,neutral,neutral,neutral,positive,neutral
3,🔥,[],positive,positive,positive,positive,positive,positive,positive
4,Lee Ronald Hall Brenda Moore Jeffrey,[],positive,neutral,neutral,neutral,neutral,positive,neutral


In [9]:
df_corrected = df.drop(columns=df.columns[2:-1])
df_corrected.head()
df_corrected.rename(columns={"sentiment_groundtruth": "sentiment"}, inplace=True)
df_corrected.head(20)

,text,drivers,sentiment
0,This why Ricc career is over... we see again t...,[Lance Stroll],negative
1,0:19,[],neutral
2,Hall Cynthia Jackson Kimberly Lewis Sharon,[Lewis Hamilton],neutral
3,🔥,[],positive
4,Lee Ronald Hall Brenda Moore Jeffrey,[],neutral
5,"here after singapore gp 2024, oh how the table...",[],negative
6,Where's the scream of the engine gone? ❤,[],neutral
7,Miller Angela Lewis Donald Davis Jason,[Lewis Hamilton],neutral
8,"I'm a Red Bull fan, so when the FIA says nothi...",[Max Verstappen],neutral
9,Red Bull Racing's last grand prix dominance in...,[],neutral


In [10]:
df_corrected.to_json("data/2024_12_03.json", orient="records")

In [18]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Ensure consistent results
DetectorFactory.seed = 0

def is_english(text):
    try:
        # Detect the language of the text
        lang = detect(text)
        return lang == 'en'
    except LangDetectException:
        # Handle cases where language detection fails
        return False
    
# df_corrected['is_english'] = df_corrected['text'].apply(is_english)
# english_dataset = df_corrected[df_corrected['is_english']]
english_dataset.drop(columns='is_english', inplace=True)
english_dataset.to_json("data/2024_12_03_only_english.json", orient="records")

/tmp/ipykernel_1469/3040149066.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_dataset.drop(columns='is_english', inplace=True)


In [19]:
print(len(english_dataset))
print(len(df_corrected))
english_dataset.head()

69493
83044


,text,drivers,sentiment
0,This why Ricc career is over... we see again t...,[Lance Stroll],negative
2,Hall Cynthia Jackson Kimberly Lewis Sharon,[Lewis Hamilton],neutral
4,Lee Ronald Hall Brenda Moore Jeffrey,[],neutral
5,"here after singapore gp 2024, oh how the table...",[],negative
6,Where's the scream of the engine gone? ❤,[],neutral
